### if you already have cpptraj input file and don't want to retype (loading traj, specify command, ...)? Use "load_cpptraj_file"

In [20]:
# test folder can be found here: https://github.com/mojyt/cpptraj/tree/master/test/Test_Comprehensive
# I just add "parm" file to `ptraj.in` and updat relative path
# TODO : more detail
!cat ptraj.in

parm ../../../test/cpptraj/Test_Comprehensive/compound.prmtop
# system: 7mer peptide in a box of
# periodic water (1577 Waters)

# Read in the trajectory file 
# starting at 1
# 100 total snapshots
# consider each one 

trajin ../../../test/cpptraj/Test_Comprehensive/trajectory.mdcrd 1 100 1

#  first calculate some things that don't depend on
#  the position of the solute in the box

# do dihedral angles

dihedral phi :1@C  :2@N  :2@CA :2@C out phi
dihedral psi   :1@N  :1@CA :1@C  :2@N out psi
dihedral omega   :1@CA :1@C  :2@N  :2@CA out omega

# calculate the distance between two atoms 
# over the 10 snapshots
# output to the file "dist.list"

distance end_to_end :1@N :7@N out  dist_end_to_end.list

#get the rms values for the whole system 
#referenced to the first snapshot

rms first :1-1584

#now deal with drift

# the peptide (residues 1->7) has drifted around 
# the box during MD.  We'd like to view the system
# with a "constant" solute.

# note that this changes the coordinates 

## lazy loading

In [21]:
from __future__ import print_function # for python 2 and 3 compat

# all I/O methods are controled in `io` module. You don't need to remmeber anything else
from pytraj import io

# load cpptraj file and return a `state`, which hold lots of information (TopologyList, DataSetList, TrajinList, ...).
# you can take the raw data from this `state`

In [22]:
# load state from `ptraj.in`
state = io.load_cpptraj_file("ptraj.in")
print (state)

from pytraj.misc import get_atts

print ("try to look for all methods/attributres in `state`")
print (get_atts(state))

try to look for all methods/attributres in `state`
['add_action', 'add_analysis', 'add_reference', 'add_trajin', 'add_trajout', 'clear_list', 'datafilelist', 'datasetlist', 'get_trajinlist', 'is_empty', 'list_all', 'remove_dataset', 'run', 'run_analyses', 'toplist', 'write_all_datafiles']


In [23]:
# see, we already load Topology file here
print (state.toplist)
print (state.toplist[0])

# perform the analysis and store data in datasetlist
state.run()

# what's in state?
# we specified "phi", "psi", "omega" dihedral calculation, end_to_end distance calculation, rmsd calculation,
# find watershell
# and even saving to file
print (state.datasetlist.get_legends())

# create alias for short-typing
dslist = state.datasetlist

Topology instance with 4854 atoms. ID = 0x2aaad3507050
['phi', 'psi', 'omega', 'end_to_end', 'RMSD_00004', 'ws[lower]', 'ws[upper]']


In [24]:
# we did save traj to disk
!ls test.mdcrd

test.mdcrd


In [30]:
# want to take the raw data out from dataset?

print ("first 10 frames, dumpt data to python `list`?")
print (dslist['phi'].tolist()[:10])

print ("")
print ("first 10 frames, dumpt data to numpy array?")
print (dslist['phi'].to_ndarray()[:10])

first 10 frames, dumpt data to python `list`?
[41.28882154215048, 51.91785128926744, 49.05564944291124, 55.63710513338503, 41.95214268721428, 47.0439840698581, 55.475476203287556, 49.372007962996335, 51.891034018639395, 59.951548994065305]

first 10 frames, dumpt data to numpy array?
[ 41.28882154  51.91785129  49.05564944  55.63710513  41.95214269
  47.04398407  55.4754762   49.37200796  51.89103402  59.95154899]


In [32]:
# want to use pandas's DataFrame?

# make a list first
table = [d0 for d0 in dslist if d0.legend in ['phi', 'psi', 'omega', 'end_to_end']]

from pytraj import to_dataframe

# display only first 11 rows
to_dataframe(table)[:10]

,end_to_end,omega,phi,psi
0,17.038716,-175.771428,41.288822,171.225967
1,16.961235,-178.535143,51.917851,174.950468
2,17.015687,-175.683427,49.055649,175.551586
3,15.707229,175.113985,55.637105,132.943782
4,16.773463,-163.859129,41.952143,144.538320
5,16.760754,-175.783954,47.043984,165.372317
6,16.245681,179.500612,55.475476,162.119407
7,17.138851,177.517108,49.372008,177.312831
8,17.502947,170.682599,51.891034,-168.635563
9,15.543892,176.942923,59.951549,160.373889


In [33]:
# bonus: `dslist` is a DataSetList object

print (get_atts(dslist))

['_add_copy_of_set', '_base_dataset_iter', 'add_existing_set', 'add_set', 'add_set_aspect', 'add_setidx', 'allocate_sets', 'clear', 'ensemble_num', 'find_coords_set', 'find_set_of_type', 'get_aspects', 'get_dataset', 'get_dtypes', 'get_legends', 'get_multiple_sets', 'get_scalar_modes', 'get_scalar_types', 'get_set', 'groupby', 'is_empty', 'keys', 'len', 'remove_set', 'set_ensemble_num', 'set_precision_of_data_sets', 'set_py_free_mem', 'size', 'to_dict', 'to_ndarray', 'tolist']


In [37]:
# want to get raw data and group them in some object holder: list, ndarray, dict?
dslist.tolist() # use tolist() for compat with `tolist` in numpy
dslist.to_ndarray()
dslist.to_dict().keys()

dict_keys(['omega', 'ws[upper]', 'end_to_end', 'RMSD_00004', 'psi', 'ws[lower]', 'phi'])